In [23]:
!pip install -q google-generativeai langchain python-dotenv pandas PyPDF2



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [24]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.prompts import PromptTemplate

In [25]:
# ✅ Load environment variables from .env
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 


In [26]:
# ✅ Configure Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('models/gemini-1.5-flash-latest')

In [27]:
# ✅ Read input text
with open("/home/fathimahusna/MCQ_Generator/data.txt", "r", encoding="utf-8") as file:
    TEXT = file.read()



In [28]:
# ✅ Constants
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}


In [29]:
# ✅ LangChain PromptTemplates (optional, for prompt formatting)
quiz_prompt_template = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template="""
Text: {text}
You are an expert MCQ maker. Create a quiz of {number} MCQs for {subject} students in a {tone} tone.
Ensure questions are based only on the text.
Use this JSON format:
{response_json}
    """
)

In [30]:
# ✅ Function to generate MCQs
def generate_quiz(text, number, subject, tone, response_json):
    prompt = quiz_prompt_template.format(
        text=text,
        number=number,
        subject=subject,
        tone=tone,
        response_json=json.dumps(response_json)
    )
    response = model.generate_content(prompt)
    return response.text

In [31]:
# ✅ Function to review MCQs
def review_quiz(subject, quiz_text):
    prompt = f"""
You are an expert English grammarian. Evaluate the following quiz for {subject}:
1. Give a 50-word complexity comment.
2. Revise MCQs if needed to match student ability.

Quiz:
{quiz_text}
    """
    response = model.generate_content(prompt)
    return response.text


In [32]:
# ✅ Function to generate and review MCQs
def generate_and_review_with_gemini(text, number, subject, tone, response_json):
    quiz = generate_quiz(text, number, subject, tone, response_json)
    review = review_quiz(subject, quiz)
    return {"quiz": quiz, "review": review}

In [33]:
#✅ Run pipeline
response = generate_and_review_with_gemini(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)

print("\n=== Generated Quiz ===\n")
print(response["quiz"])

print("\n=== Review Feedback ===\n")
print(response["review"])


=== Generated Quiz ===

```json
{
  "1": {
    "mcq": "Which of the following is NOT a method mentioned in the text for maintaining sustainable food production?",
    "options": {
      "a": "Production of high-yielding varieties of plants and animals",
      "b": "Development of drought-resistant crop strains",
      "c": "Production of disease-resistant plants and animal varieties",
      "d": "Improvement of post-harvest technological methods"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "According to the text, what is the primary reason understanding plant life is crucial?",
    "options": {
      "a": "To understand plant reproduction",
      "b": "To increase food productivity in response to growing human population",
      "c": "To study plant genetics",
      "d": "To understand plant evolution"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "Which of the following is NOT listed as a characteristic of living organisms in the text?",
    "options": {
      "a": "Metab

In [34]:
def clean_gemini_output(text):
    if text.strip().startswith("```json"):
        text = text.strip()[7:]  # remove ```json
    if text.strip().endswith("```"):
        text = text.strip()[:-3]  # remove ending ```
    return text.strip()

In [35]:
# ✅ Save to CSV if possible
try:
    quiz_text = clean_gemini_output(response["quiz"])
    quiz_json = json.loads(quiz_text)
    quiz_table_data = []
    for key, value in quiz_json.items():
        mcq = value["mcq"]
        options = " | ".join([f"{k}: {v}" for k, v in value["options"].items()])
        correct = value["correct"]
        quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

    df = pd.DataFrame(quiz_table_data)
    df.to_csv("generated_mcqs.csv", index=False)
    print("\n✅ MCQs saved to generated_mcqs.csv")
except Exception as e:
    print("\n❌ Could not parse quiz as JSON:", e)



✅ MCQs saved to generated_mcqs.csv


In [ ]:
# # ✅ Install dependencies (if not already installed)
# # !pip install -q google-generativeai langchain python-dotenv pandas PyPDF2

# import os
# import json
# import pandas as pd
# from dotenv import load_dotenv
# import google.generativeai as genai
# from langchain.prompts import PromptTemplate

# # ✅ Load environment variables from .env
# load_dotenv()
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or "your-gemini-api-key"

# # ✅ Configure Gemini API
# genai.configure(api_key=GOOGLE_API_KEY)
# model = genai.GenerativeModel("gemini-pro")

# # ✅ Read input text
# with open("data.txt", "r", encoding="utf-8") as file:
#     TEXT = file.read()

# # ✅ Constants
# NUMBER = 5
# SUBJECT = "biology"
# TONE = "simple"

# RESPONSE_JSON = {
#     "1": {
#         "mcq": "multiple choice question",
#         "options": {
#             "a": "choice here",
#             "b": "choice here",
#             "c": "choice here",
#             "d": "choice here"
#         },
#         "correct": "correct answer"
#     }
# }

# # ✅ LangChain PromptTemplates (optional, for prompt formatting)
# quiz_prompt_template = PromptTemplate(
#     input_variables=["text", "number", "subject", "tone", "response_json"],
#     template="""
# Text: {text}
# You are an expert MCQ maker. Create a quiz of {number} MCQs for {subject} students in a {tone} tone.
# Ensure questions are based only on the text.
# Use this JSON format:
# {response_json}
#     """
# )

# def generate_quiz(text, number, subject, tone, response_json):
#     prompt = quiz_prompt_template.format(
#         text=text,
#         number=number,
#         subject=subject,
#         tone=tone,
#         response_json=json.dumps(response_json)
#     )
#     response = model.generate_content(prompt)
#     return response.text

# def review_quiz(subject, quiz_text):
#     prompt = f"""
# You are an expert English grammarian. Evaluate the following quiz for {subject}:
# 1. Give a 50-word complexity comment.
# 2. Revise MCQs if needed to match student ability.

# Quiz:
# {quiz_text}
#     """
#     response = model.generate_content(prompt)
#     return response.text

# def generate_and_review_with_gemini(text, number, subject, tone, response_json):
#     quiz = generate_quiz(text, number, subject, tone, response_json)
#     review = review_quiz(subject, quiz)
#     return {"quiz": quiz, "review": review}

# # ✅ Run pipeline
# response = generate_and_review_with_gemini(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)

# print("\n=== Generated Quiz ===\n")
# print(response["quiz"])

# print("\n=== Review Feedback ===\n")
# print(response["review"])

# # ✅ Save to CSV if possible
# try:
#     quiz_json = json.loads(response["quiz"])
#     quiz_table_data = []
#     for key, value in quiz_json.items():
#         mcq = value["mcq"]
#         options = " | ".join([f"{k}: {v}" for k, v in value["options"].items()])
#         correct = value["correct"]
#         quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

#     df = pd.DataFrame(quiz_table_data)
#     df.to_csv("generated_mcqs.csv", index=False)
#     print("\n✅ MCQs saved to generated_mcqs.csv")
# except Exception as e:
#     print("\n❌ Could not parse quiz as JSON:", e)
